# 9.2 再谈向量类

In [3]:
from array import array
import math

class Vector2d:
    typecode = 'd'  # 类属性，在实例和字节序列之间转换时使用

    def __init__(self, x, y) -> None:
        self.x = float(x)
        self.y = float(y)
    
    def __iter__(self): # 将实例变成可迭代的对象
        return (i for i in (self.x, self.y))
    
    def __repr__(self) -> str:
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)   # 用 !r 获取各个分量的表示形式，因为实例是可迭代对象，所以 *self 会把 x 和 y 分量提供给 format 函数

    def __str__(self) -> str:
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
                bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))

In [6]:
v1 = Vector2d(3,4)
print(v1.x, v1.y)
x, y = v1
print(x,y)
print(v1)
v1

3.0 4.0
3.0 4.0
(3.0, 4.0)


Vector2d(3.0, 4.0)

In [9]:
v1_clone = eval(repr(v1))
v1 == v1_clone

True

In [11]:
octets = bytes(v1)
octets

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [13]:
(bytes([ord(v1.typecode)]) + bytes(array(v1.typecode, v1)))

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [20]:
bytes(array(v1.typecode, v1))

b'\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

# 9.3 备选构造方法
我们可以将 Vector2d 实例转换成字节序列了，也应该能从字节序列转换成 Vector2d 实例。

In [1]:
from array import array
import math

class Vector2d:
    typecode = 'd'  # 类属性，在实例和字节序列之间转换时使用

    def __init__(self, x, y) -> None:
        self.x = float(x)
        self.y = float(y)
    
    def __iter__(self): # 将实例变成可迭代的对象
        return (i for i in (self.x, self.y))
    
    def __repr__(self) -> str:
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)   # 用 !r 获取各个分量的表示形式，因为实例是可迭代对象，所以 *self 会把 x 和 y 分量提供给 format 函数

    def __str__(self) -> str:
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
                bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

In [33]:
octets = bytes(v1)
octets

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [38]:
v3 = Vector2d.frombytes(octets)
v3

Vector2d(3.0, 4.0)

# 9.4 classmethod 与 staticmethod
classmethod：定义操作类，而不是操作实例的方法。它改变了调用方法的方式，因此类方法的第一个参数是类本身，而不是实例。它最常见的用途是定义备选构造方法，按照约定，类方法的第一个参数名为 cls  
staticmethod：它也会改变方法的调用方式，但第一个参数不是特殊的值。其实静态方法就是普通的函数，只是碰巧在类的定义体中，而不是在模块层定义

In [39]:
class Demo:
    @classmethod
    def klassmeth(*args):
        return args
    @staticmethod
    def statmeth(*args):
        return args


Demo.klassmeth

<bound method Demo.klassmeth of <class '__main__.Demo'>>

In [41]:
# 不管怎样调用 Demo.klassmeth，它的第一个参数始终是 Demo 类
Demo.klassmeth('a')

(__main__.Demo, 'a')

In [40]:
Demo.statmeth

<function __main__.Demo.statmeth(*args)>

In [43]:
# Demo.statmeth 的行为与普通函数相似
Demo.statmeth('a')

('a',)

# 9.5 格式化显示
内置的 format() 函数和 str.format() 方法把各个类型的格式化方式委托给相应的 .\_\_format__(format_spec) 方法。format_spec 是格式说明符，它是
* format(my_obj, format_spec) 的第二个参数，或者
* str.format() 方法里的格式字符串，{} 里代换字段中冒号后面的部分

如果类没有定义 \_\_format__ 方法，从 object 继承的方法会返回 str(my_obj)，我们为 Vector2d 类定义了 \_\_str__ 方法，因此可以这么做：

In [2]:
v1 = Vector2d(3, 4)
format(v1)

'(3.0, 4.0)'

但这种方式无法传入格式说明符，所以需要自行实现 \_\_format__ 方法

In [6]:
from array import array
import math

class Vector2d:
    typecode = 'd'  # 类属性，在实例和字节序列之间转换时使用

    def __init__(self, x, y) -> None:
        self.x = float(x)
        self.y = float(y)
    
    def __iter__(self): # 将实例变成可迭代的对象
        return (i for i in (self.x, self.y))
    
    def __repr__(self) -> str:
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)   # 用 !r 获取各个分量的表示形式，因为实例是可迭代对象，所以 *self 会把 x 和 y 分量提供给 format 函数

    def __str__(self) -> str:
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
                bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)
    
    # 实现 format(self)
    def __format__(self, format_spec: str = '') -> str:
        components = (format(c, format_spec) for c in self)
        return '({}, {})'.format(*components)

In [7]:
v1 = Vector2d(3, 4)
format(v1)

'(3.0, 4.0)'

In [8]:
format(v1, '.2f')

'(3.00, 4.00)'

下面要在微语言中添加一个自定义的格式代码：如果格式符以 'p' 结尾，那么在极坐标中显示向量，即 <r, θ>

In [13]:
class Vector2d:
    typecode = 'd'  # 类属性，在实例和字节序列之间转换时使用

    
    # 首先需要定义一个简单的 angle 方法来计算角度
    def angle(self):
        return math.atan2(self.y, self.x)

    
    # 增强的 __format__ 方法
    def __format__(self, format_spec: str = '') -> str:
        if format_spec.endswith('p'):
            format_spec = format_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, format_spec) for c in coords)
        return outer_fmt.format(*components)

    def __init__(self, x, y) -> None:
        self.x = float(x)
        self.y = float(y)
    
    def __iter__(self): # 将实例变成可迭代的对象
        return (i for i in (self.x, self.y))
    
    def __repr__(self) -> str:
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)   # 用 !r 获取各个分量的表示形式，因为实例是可迭代对象，所以 *self 会把 x 和 y 分量提供给 format 函数

    def __str__(self) -> str:
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
                bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)
    


In [14]:
format(Vector2d(1, 1), 'p')

'<1.4142135623730951, 0.7853981633974483>'

In [15]:
format(Vector2d(1, 1), '.3ep')

'<1.414e+00, 7.854e-01>'

In [16]:
format(Vector2d(1, 1), '.5fp')

'<1.41421, 0.78540>'

In [17]:
format(Vector2d(1, 1), '.5f')

'(1.00000, 1.00000)'

# 9.6 可散列的 Vector2d
为了将 Vector2d 实例变成可散列的，必须使用 \_\_hash__ 方法(还需要 \_\_eq__ 方法，已经实现了)，此外，还要让向量不可变

In [18]:
class Vector2d:
    typecode = 'd'  # 类属性，在实例和字节序列之间转换时使用

    # 使用两个前导下划线将属性标记为私有的
    def __init__(self, x, y) -> None:
        self.__x = float(x)
        self.__y = float(y)
    
    # property 装饰器将读值方法标记为特性
    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y
    
    # 向量不可变后才能实现 __hash__ 方法。它需要返回一个整数，理想情况下要考虑对象属性的散列值。因为相等的对象应该具有相同的散列值。
    # 根据特殊方法 __hash__ 的文档，最好使用运算符异或 (^) 混合各分量的散列值
    def __hash__(self) -> int:
        return hash(self.x) ^ hash(self.y)
        


In [19]:
# 完整代码
class Vector2d:
    typecode = 'd'  # 类属性，在实例和字节序列之间转换时使用

    # 使用两个前导下划线将属性标记为私有的
    def __init__(self, x, y) -> None:
        self.__x = float(x)
        self.__y = float(y)
    
    # property 装饰器将读值方法标记为特性
    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y
    
    # 向量不可变后才能实现 __hash__ 方法。它需要返回一个整数，理想情况下要考虑对象属性的散列值。因为相等的对象应该具有相同的散列值。
    # 根据特殊方法 __hash__ 的文档，最好使用运算符异或 (^) 混合各分量的散列值
    def __hash__(self) -> int:
        return hash(self.x) ^ hash(self.y)

   
    def angle(self):
        return math.atan2(self.y, self.x)

    
    def __format__(self, format_spec: str = '') -> str:
        if format_spec.endswith('p'):
            format_spec = format_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, format_spec) for c in coords)
        return outer_fmt.format(*components)
    
    def __iter__(self): # 将实例变成可迭代的对象
        return (i for i in (self.x, self.y))
    
    def __repr__(self) -> str:
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)   # 用 !r 获取各个分量的表示形式，因为实例是可迭代对象，所以 *self 会把 x 和 y 分量提供给 format 函数

    def __str__(self) -> str:
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
                bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)
    

In [20]:
v1 = Vector2d(3, 4)
v2 = Vector2d(3.1, 4.2)
hash(v1), hash(v2)

(7, 384307168202284039)

In [21]:
set([v1, v2])

{Vector2d(3.0, 4.0), Vector2d(3.1, 4.2)}

要想创建可散列的类型，不一定要实现特性，也不一定需要保护实例属性。只需正确地实现 \_\_hash__ 和 \_\_eq__ 方法。但是，实例的散列值绝不应该变化。

如果定义的类型有标量数值，可能还需要实现 \_\_int__ 和 \_\_float__ 方法，被 int() 和 float() 构造函数调用，以便在某些情况下用于强制转换类型。此外，还有用于支持内置 complex() 构造函数的 \_\_complex__ 方法。

# 9.7 Python 私有属性和 “受保护的” 属性
如果以 \_\_mood 的形式命名实例属性，Python 会把属性名存入实例的 \_\_dict__ 属性中，并且会在前面加上一个下划线和类名。  
因此，对于 Dog 类来说 \_\_mood 会变成 \_Dog\_\_mood，这个语言特性叫 _名称改写_

In [22]:
v1 = Vector2d(3, 4)
v1.__dict__

{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}

# 9.8 使用 \_\_solts__ 类节省空间
默认情况下，Python 在各个实例中名为 \_\_dict__ 的字典里存储实例属性。为了使用底层的散列表提升访问速度，字典会消耗大量内存。  
而通过 \_\_solts__ 类属性，能节省大量内存，方法是让解释器在元组中储存实例属性，而不用字典。  
继承自超类的 \_\_solts__ 属性没有效果。Python 只会使用各个类中定义的 \_\_solts__ 属性。

In [ ]:
class Vector2d:
    __solts__ = ('__x', '__y')

    typecode = 'd'

    # 省略

\_\_solt__ 的问题：
* 每个子类都要定义 \_\_solt__ 属性，因为解释器会忽略继承的 \_\_solt__ 属性
* 实例只能拥有 \_\_solt__ 中列出的时候，除非把 '\_\_dict__' 加入 \_\_solt__ 中，但这样会失去节省内存的功效
* 如果不把 '\_\_weakref__' 加入 \_\_solt__，实例就不能作为弱引用的目标

# 9.9 覆盖类属性
Python 有一个很独特的特效，类属性可用于为实例属性提供默认值。Vector2d 中有个 typecode 类属性，\_\_bytes__ 方法两次用到了它，且都是使用 self.typecode 读取。因为 Vector2d 实例本身没有 typecode 属性，所以 self.typecode 默认获取的是 Vector2d.typecode 类属性的值  
但是，如果为不存在的实例属性复制，会新建实例属性。加入我们为 typecode 实例属性复制，那么同名类属性不受影响。但自此之后实例读取的 self.typecode 是实例属性 typecode，也就是把同名类属性覆盖了。借助这一特性，可以为各个实例的 typecode 属性定制不同的值。  
Vector2d.typecode 属性默认值是 'd'，即转换成字节序列使用 8 字节双精度浮点数表示向量的各个分量。如果在转换之前把 Vector2d 实例的 typecode 属性设为 'f'，那么使用 4 字节单精度浮点数表示各个分量。

In [23]:
from array import array
import math

class Vector2d:
    typecode = 'd'  # 类属性，在实例和字节序列之间转换时使用

    def __init__(self, x, y) -> None:
        self.x = float(x)
        self.y = float(y)
    
    def __iter__(self): # 将实例变成可迭代的对象
        return (i for i in (self.x, self.y))
    
    def __repr__(self) -> str:
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)   # 用 !r 获取各个分量的表示形式，因为实例是可迭代对象，所以 *self 会把 x 和 y 分量提供给 format 函数

    def __str__(self) -> str:
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
                bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))

In [25]:
v1 = Vector2d(1.1,2.2)
dumpd = bytes(v1)
dumpd

b'd\x9a\x99\x99\x99\x99\x99\xf1?\x9a\x99\x99\x99\x99\x99\x01@'

In [26]:
len(dumpd)

17

In [27]:
v1.typecode = 'f'
dumpf = bytes(v1)
dumpf

b'f\xcd\xcc\x8c?\xcd\xcc\x0c@'

In [28]:
Vector2d.typecode

'd'

如果想修改类属性的值，必须在类上修改而不是通过实例。

In [ ]:
# 修改类属性的值
Vector2d.typecode = 'f'

In [30]:
# ShortVector2d 是 Vector2d 的子类，只用于覆盖 typecode 的默认值
class ShortVector2d(Vector2d):
    typecode = 'f'

sv = ShortVector2d(1/11, 1/27)
print(sv)
bytes(sv)

(0.09090909090909091, 0.037037037037037035)


b'f\x8c.\xba=&\xb4\x17='

这也说明了我们在 Vector2d.\_\_repr__ 中为什么没有硬编吗 class_name 的值，而是使用 type(self)\_\_name__ 获取：为了支持继承